# Peek into the dataset
* https://www.kaggle.com/eduardo4jesus/stanford-cars-dataset-a-quick-look-up
* https://www.kaggle.com/jutrera/training-a-densenet-for-the-stanford-car-dataset

Make sure you:
1. Load the dataset correctly.

In [1]:
# TODO: check against https://github.com/johntd54/stanford_car
from dv.MyImageFolderWithPaths import *
from utils.transform import *

img_dir = '/Users/kevinsiswandi/dvfp/mDFL-CNN/dataset'
# transformations are defined in "dv" module
transform_train = get_transform_for_train()
transform_test  = get_transform_for_test()
transform_test_simple = get_transform_for_test_simple()

train_dataset = CarsDataset(os.path.join(img_dir,'devkit/cars_train_annos.mat'),
                        os.path.join(img_dir,'cars_train'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_train
                        )

test_dataset = CarsDataset(os.path.join(img_dir,'devkit/cars_test_annos_withlabels.mat'),
                        os.path.join(img_dir,'cars_test'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_test
                        )

test_dataset_simple = CarsDataset(os.path.join(img_dir,'devkit/cars_test_annos_withlabels.mat'),
                        os.path.join(img_dir,'cars_test'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_test_simple
                        )


In [2]:
len(train_dataset.car_names)


196

In [5]:
lst = set()
for i in range(train_dataset.__len__()):
    lst.add(train_dataset.car_annotations[i][-2][0][0])